In [ ]:
!unzip "/content/drive/My Drive/GP/img_align_celeba.zip"  -d "/content/data"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/jesse1029/Fake-Face-Images-Detection-Tensorflow.git

Cloning into 'Fake-Face-Images-Detection-Tensorflow'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 67 (delta 13), reused 19 (delta 6), pack-reused 35
Unpacking objects: 100% (67/67), done.


In [ ]:
import sys
sys.path.append("/content/drive/My Drive/GP")
sys.path.append("/content/Fake-Face-Images-Detection-Tensorflow")

In [ ]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import glob
import utils
import traceback
import numpy as np
import tensorflow as tf
import models_64x64 as models


""" param """
epoch = 50
batch_size = 64
lr = 0.0002
z_dim = 100
n_critic = 5
gpu_id = 3

''' data '''
# you should prepare your own data in ./data/img_align_celeba
# celeba original size is [218, 178, 3]


def preprocess_fn(img):
    crop_size = 108
    re_size = 64
    img = tf.image.crop_to_bounding_box(img, (218 - crop_size) // 2, (178 - crop_size) // 2, crop_size, crop_size)
    img = tf.to_float(tf.image.resize_images(img, [re_size, re_size], method=tf.image.ResizeMethod.BICUBIC)) / 127.5 - 1
    return img

img_paths = glob.glob('/content/data/img_align_celeba/')
data_pool = utils.DiskImageData('/content/data/img_align_celeba/', batch_size, shape=[218, 178, 3], preprocess_fn=preprocess_fn)
print(len(data_pool))

""" graphs """
tf.reset_default_graph() 
with tf.device('/gpu:%d' % gpu_id):
    ''' models '''
    generator = models.generator
    discriminator = models.discriminator_wgan_gp

    ''' graph '''
    # inputs
    real = tf.placeholder(tf.float32, shape=[None, 64, 64, 3])
    z = tf.placeholder(tf.float32, shape=[None, z_dim])

    # generate
    fake = generator(z, reuse=False)

    # dicriminate
    r_logit = discriminator(real, reuse=False)
    f_logit = discriminator(fake)
    # losses
    def gradient_penalty(real, fake, f):
        def interpolate(a, b):
            shape = tf.concat((tf.shape(a)[0:1], tf.tile([1], [a.shape.ndims - 1])), axis=0)
            alpha = tf.random_uniform(shape=shape, minval=0., maxval=1.)
            inter = a + alpha * (b - a)
            inter.set_shape(a.get_shape().as_list())
            return inter

        x = interpolate(real, fake)
        pred = f(x)
        gradients = tf.gradients(pred, x)[0]
        slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients),reduction_indices= [y for y in range(1, x.shape.ndims)]))
        gp = tf.reduce_mean((slopes - 1.)**2)
        return gp
        
    wd = tf.reduce_mean(r_logit) - tf.reduce_mean(f_logit)
    
    gp = gradient_penalty(real, fake, discriminator)
    d_loss = -wd + gp * 10.0
    g_loss = -tf.reduce_mean(f_logit)

    # otpims
    d_var = utils.trainable_variables('discriminator')
    g_var = utils.trainable_variables('generator')
    d_step = tf.train.AdamOptimizer(learning_rate=lr, beta1=0.5).minimize(d_loss, var_list=d_var)
    g_step = tf.train.AdamOptimizer(learning_rate=lr, beta1=0.5).minimize(g_loss, var_list=g_var)

    # summaries
    d_summary = utils.summary({wd: 'wd', gp: 'gp'})
    g_summary = utils.summary({g_loss: 'g_loss'})

    # sample
    f_sample = generator(z, training=False)

""" train """
''' init '''
# session
sess = utils.session()
# iteration counter
it_cnt, update_cnt = utils.counter()
# saver
saver = tf.train.Saver(max_to_keep=5)
# summary writer
summary_writer = tf.summary.FileWriter('/content/drive/My Drive/GP/wpg_gp/summaries/celeba_wgan_gp', sess.graph)

''' initialization '''
ckpt_dir = '/content/drive/My Drive/GP/wpg_gp/checkpoints/celeba_wgan_gp'
#utils.mkdir(ckpt_dir + '/')
if not utils.load_checkpoint(ckpt_dir, sess):
    sess.run(tf.global_variables_initializer())

''' train '''
try:
    z_ipt_sample = np.random.normal(size=[100, z_dim])

    batch_epoch = len(data_pool) // (batch_size * n_critic)
    max_it = epoch * batch_epoch
    for it in range(sess.run(it_cnt), max_it):
        sess.run(update_cnt)

        # which epoch
        epoch = it // batch_epoch
        it_epoch = it % batch_epoch + 1

        # train D
        for i in range(n_critic):
            # batch data
            real_ipt = data_pool.batch()
            z_ipt = np.random.normal(size=[batch_size, z_dim])
            d_summary_opt, _ = sess.run([d_summary, d_step], feed_dict={real: real_ipt, z: z_ipt})
        summary_writer.add_summary(d_summary_opt, it)

        # train G
        z_ipt = np.random.normal(size=[batch_size, z_dim])
        g_summary_opt, _ = sess.run([g_summary, g_step], feed_dict={z: z_ipt})
        summary_writer.add_summary(g_summary_opt, it)

        # display
        if it % 1 == 0:
            print("Epoch: (%3d) (%5d/%5d)" % (epoch, it_epoch, batch_epoch))

        # save
        if (it + 1) % 1000 == 0:
            save_path = saver.save(sess, '%s/Epoch_(%d)_(%dof%d).ckpt' % (ckpt_dir, epoch, it_epoch, batch_epoch))
            print('Model saved in file: % s' % save_path)

        # sample
        if (it + 1) % 100 == 0:
            f_sample_opt = sess.run(f_sample, feed_dict={z: z_ipt_sample})

            save_dir = '/content/drive/My Drive/GP/wpg_gp/sample_images_while_training/celeba_wgan_gp'
            utils.mkdir(save_dir + '/')
            utils.imwrite(utils.immerge(f_sample_opt, 10, 10), '%s/Epoch_(%d)_(%dof%d).jpg' % (save_dir, epoch, it_epoch, batch_epoch))

except  Exception as e:
    traceback.print_exc()
finally:
    print(" [*] Close main session!")
    sess.close()


In [ ]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import glob
import utils
import traceback
import numpy as np
import tensorflow as tf
import models_64x64 as models
import imageio
'''GENERATION'''
tf.reset_default_graph() 
def to_range(images, min_value=0.0, max_value=1.0, dtype=None):
    """
    transform images from [-1.0, 1.0] to [min_value, max_value] of dtype
    """
    assert \
        np.min(images) >= -1.0 - 1e-5 and np.max(images) <= 1.0 + 1e-5 \
        and (images.dtype == np.float32 or images.dtype == np.float64), \
        'The input images should be float64(32) and in the range of [-1.0, 1.0]!'
    if dtype is None:
        dtype = images.dtype
    return ((images + 1.) / 2. * (max_value - min_value) + min_value).astype(dtype)


def batchimwrite(image, path, cnt,prob):
    """ save an [-1.0, 1.0] image """
    for i in range(image.shape[0]):
        image = np.array(image, copy=True)
        imageio.imwrite("%s_%d.jpg" % (path, cnt) , to_range(image[i, :, :, :], 0, 255, np.uint8))
        print(cnt)
        print(prob[i])
        cnt += 1
    return cnt

""" param """
epoch = 50
batch_size = 100
lr = 0.0002
z_dim = 100
gpu_id = 3

''' data '''
# you should prepare your own data in ./data/img_align_celeba
# celeba original size is [218, 178, 3]
tf.reset_default_graph()

""" graphs """
with tf.device('/gpu:%d' % gpu_id):
    ''' models '''
    generator = models.generator
    discriminator = models.discriminator_wgan_gp

    ''' graph '''
    # inputs
    z = tf.placeholder(tf.float32, shape=[None, z_dim])

    # generate
    fake = generator(z, training=False ,reuse=False)
    f_logit = discriminator(fake,training=False ,reuse=False)


""" train """
''' init '''
# session
sess = utils.session()
# saver
saver = tf.train.Saver()

''' initialization '''
ckpt_dir = '/content/drive/My Drive/GP/wpg_gp/checkpoints/celeba_wgan_gp/Epoch_(34)_(478of633).ckpt'
saver.restore(sess, ckpt_dir)
sess.run(tf.global_variables_initializer())
saver.restore(sess, ckpt_dir)
print("hello")

try:
    
    cnt = 0
    while(True):
        if(cnt >= 100):
          break
        s= int(cnt/10000)
        z_ipt = np.random.normal(size=[batch_size, z_dim])
        img = sess.run(fake, feed_dict={z: z_ipt})
        prob = sess.run(f_logit,{z: z_ipt})
        #save_dir = '/content/drive/My Drive/GP/wpg_gp/results_images/celeba_wgan_gp'
        save_dir = '/celeba_wgan_gp'+str(s)
        utils.mkdir(save_dir + '/')
        cnt = batchimwrite(img, '%s/img' % (save_dir), cnt, prob)


except Exception as e:
    traceback.print_exc()
finally:
    print(" [*] Close main session!")
    sess.close()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.flatten instead.


INFO:tensorflow:Restoring parameters from /content/drive/My Drive/GP/wpg_gp/checkpoints/celeba_wgan_gp/Epoch_(34)_(478of633).ckpt
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/GP/wpg_gp/checkpoints/celeba_wgan_gp/Epoch_(34)_(478of633).ckpt
hello
0
[51.05012]
1
[49.946712]
2
[49.145004]
3
[50.96952]
4
[49.80316]
5
[51.750042]
6
[47.97268]
7
[50.56752]
8
[46.33333]
9
[44.367718]
10
[51.69427]
11
[50.032917]
12
[48.529823]
13
[47.824715]
14
[49.346817]
15
[48.247635]
16
[45.79146

In [ ]:
!zip -r '/content/wgan_image0.zip' '/celeba_wgan_gp0/'


  adding: celeba_wgan_gp0/ (stored 0%)
  adding: celeba_wgan_gp0/img_2.jpg (deflated 8%)
  adding: celeba_wgan_gp0/img_19.jpg (deflated 7%)
  adding: celeba_wgan_gp0/img_56.jpg (deflated 8%)
  adding: celeba_wgan_gp0/img_55.jpg (deflated 8%)
  adding: celeba_wgan_gp0/img_69.jpg (deflated 9%)
  adding: celeba_wgan_gp0/img_98.jpg (deflated 7%)
  adding: celeba_wgan_gp0/img_32.jpg (deflated 9%)
  adding: celeba_wgan_gp0/img_12.jpg (deflated 8%)
  adding: celeba_wgan_gp0/img_34.jpg (deflated 9%)
  adding: celeba_wgan_gp0/img_31.jpg (deflated 7%)
  adding: celeba_wgan_gp0/img_53.jpg (deflated 7%)
  adding: celeba_wgan_gp0/img_99.jpg (deflated 8%)
  adding: celeba_wgan_gp0/img_74.jpg (deflated 7%)
  adding: celeba_wgan_gp0/img_5.jpg (deflated 6%)
  adding: celeba_wgan_gp0/img_10.jpg (deflated 7%)
  adding: celeba_wgan_gp0/img_62.jpg (deflated 7%)
  adding: celeba_wgan_gp0/img_43.jpg (deflated 9%)
  adding: celeba_wgan_gp0/img_4.jpg (deflated 7%)
  adding: celeba_wgan_gp0/img_79.jpg (deflated

In [ ]:
!zip -r '/content/wgan_image11.zip' '/celeba_wgan_gp11/'

Streaming output truncated to the last 5000 lines.
  adding: celeba_wgan_gp11/img_113252.jpg (deflated 7%)
  adding: celeba_wgan_gp11/img_116996.jpg (deflated 7%)
  adding: celeba_wgan_gp11/img_112185.jpg (deflated 9%)
  adding: celeba_wgan_gp11/img_115975.jpg (deflated 9%)
  adding: celeba_wgan_gp11/img_111704.jpg (deflated 8%)
  adding: celeba_wgan_gp11/img_113790.jpg (deflated 7%)
  adding: celeba_wgan_gp11/img_113439.jpg (deflated 12%)
  adding: celeba_wgan_gp11/img_116936.jpg (deflated 7%)
  adding: celeba_wgan_gp11/img_116695.jpg (deflated 7%)
  adding: celeba_wgan_gp11/img_110377.jpg (deflated 7%)
  adding: celeba_wgan_gp11/img_110840.jpg (deflated 7%)
  adding: celeba_wgan_gp11/img_115938.jpg (deflated 7%)
  adding: celeba_wgan_gp11/img_111567.jpg (deflated 7%)
  adding: celeba_wgan_gp11/img_118739.jpg (deflated 7%)
  adding: celeba_wgan_gp11/img_117209.jpg (deflated 9%)
  adding: celeba_wgan_gp11/img_111925.jpg (deflated 8%)
  adding: celeba_wgan_gp11/img_113797.jpg (deflated 